### File description
Use GPT-4 to annotate data for training. 

Given an input story, statement and principle, we use GPT4 to provide an explanation for whether the principle is broken - and why. We use the same data-samples (input story + statement)-pairs as the fully manual annotated data.

Output is stored in data/GPT4_full.json.
The json file contains fields 'explanation_and_answer', 'answer', 'input' (story+statement), and 'question'

### Remarks about generating the data with GPT4
These remarks were made by analysing a random 20 samples of the dataset.

- To start all coherence says "no nothing wrong" (4/4 - my annotations said 2/4). I believe this is a cause for concern, so rephrase the condition/question to be a bit more specific -> it catches one contradiction (that I did not catch in my annotations).
- Suffering was also not very well-phrased (effect of 'don't consider emotions' had opposite of wanted effect)
- Running twice with standard temperature gave different explanations AND yes/no results (4 changes)! Also a temperature of 0.5 -> changing answers. So we lower the temperature to 0.1. And still have good explanations.
- 75% of yes/no after explanations match my annotations. Only 50% of yes/no without explaining first. So we will use the yes/no labels from the explanations.
- I would still say GPT-4 is not always correct. Eg this explanation:

        The input story does not mention any mental illness or disability that the defendant might have. However, the statement mentions that the defendant believes he is beginning to develop a mental illness. The quote from the statement is: "I believe I am beginning to develop a mental illness." Therefore, the statement does not fail to mention any mental illness or disability. <no>

## The generation

In [2]:
from key import test_key, miri_key
from info import principles, whatCR, rewriteR, yesnoCR_turned, g4_yesno
import numpy as np
import openai
import time
import pandas as pd
from datasets import load_dataset

openai.api_key = miri_key
yesnoCR = yesnoCR_turned
data_path = "/home/vsb29/rds/hpc-work/project/data/data_file1.json"
data = load_dataset("json", data_files=data_path)
data

Found cached dataset json (/home/vsb29/.cache/huggingface/datasets/json/default-ba6b1c046a561edb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input'],
        num_rows: 281
    })
})

In [7]:
yesnoCR_to_int = {
    """Is anything in the statement badly phrased or difficult to follow?""": 0,
    """Does the statement ignore the importance of the defendant's guilty plea (an important mitigating factor)?""": 1,
    """Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? We don't consider emotional stress or what may happen in the future as suffering.""": 2,
    """Does the statement break the rule 'only mention the defendant's age if it is outside of the interval 30-79'?""": 3,
    """Does the statement and input story seem to disagree on whether the defendant has a prior criminal record?""": 4,
    """Does the statement fail to mention any mental illness or disability (if the defendant has this)?""": 5,
    """Does the statement miss or add important parts of explaining the offense (and the defendant's involvement in it), compared to the input story?""": 6,
    """Does the statement give a different impression than the input story of whether the defendant has or want to make amends or 'make up' for the crime?""": 7,
    """Does the statement give a different impression than the input story of whether the defendant has or wants to self-improve (eg counselling) or prevent the offense from happening again?""": 8,
    """Does the statement give a different impression than the input story of the defendant's good character (for example contributing to the community in some way)?""": 9,
    """Does the statement contain any misplaced information for a plea in mitigation (eg mentioning the 'input story' or another statement)?""": 10,
    """Does the statement overemphasize, or show emotions different from the input story?""": 11,
    """Is there any information in the statement that is unsupported by the input story?""": 12
}

In [8]:
question = data['train'][0]['instruction'].split(" Please answer 'yes' or 'no'.")[0]
print(question)
print(yesnoCR_to_int[question])
print(g4_yesno[yesnoCR_to_int[question]])

Does the statement overemphasize, or show emotions different from the input story?
11
Does the statement overemphasize, or show emotions different from the input story?


#### yes/no data with explanation

In [9]:
explanations_A = []

In [20]:
sys_prompt = "You will be presented with a question, and then an input story and a statement. You must analyse the input story and statement in order to properly answer the question."
yesnoWhy = " Start by thinking about the question step by step, and include short relevant quotes from the input story and from the statement. Then always end by answering the question concisely with <yes> or <no> being the last word."

diff = " Make sure to think carefully about what is said in the input story VS what is said in the statement."

f = open("GPT4_explainYN", "a") # Temporary file that will be deleted, but is a safeguard if anything crashes during the loop
for j in range(281):
    time.sleep(1)
    print(j)
    f.write("\n###"+str(j)+"\n")
    input_story, initial_statement = data['train'][j]['input'].split("\n### STATEMENT: ")
    question = data['train'][j]['instruction'].split(" Please answer 'yes' or 'no'.")[0] # question typically asked Vicuna/Alpaca
    # if princ == "Is anything in the statement badly phrased or difficult to follow?":
    #     princ = "Is anything in the statement badly phrased, too repetative, self-contradictory, or otherwise unreasonable?"
    # elif princ == "Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? We don't consider emotional stress or what may happen in the future as suffering.":
    #     princ = " Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? (Emotional stress should automatically trigger a <yes> here)."
    # elif princ == "Does the statement break the rule 'only mention the defendant's age if it is outside of the interval 30-79'?":
    #     princ = "Does the statement break the rule 'only mention the defendant's age if it is OUTSIDE of the interval 30-79'? Think step by step."
    g4_q = g4_yesno[yesnoCR_to_int[question]] # Modified prompt/question for GPT-4

    chat_log = [{"role": "system", "content": sys_prompt},
                {"role": "user", "content": "### QUESTION: " + g4_q + yesnoWhy + diff},
                {"role": "user", "content": input_story + "\n### STATEMENT: " + initial_statement + "\n### ANSWER:"}]

    try:
        explanation = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=chat_log,
                temperature=0.1
            )
    except:
        print("Failed request; we wait.")
        time.sleep(20)
        explanation = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=chat_log,
                temperature=0.1
            )
    explanations_A.append(explanation['choices'][0]['message']['content'].replace("\n", ""))
    f.write(explanation["choices"][0]["message"]["content"].replace("\n", ""))
f.close()


200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
Failed request; we wait.
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


In [21]:
answer = []
for i in range(len(explanations_AE)):
    last = explanations_AE[i][-5:]
    if 'yes' in last.lower():
        answer.append('yes')
    elif 'no' in last.lower():
        answer.append('no')
    else:
        print(last)
print(answer)
print(len(answer))

['no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'

In [22]:
data_dict = {'explanation_and_answer': explanations_AE, 'answer': answer, 'input': data['train']['input'], 'question': data['train']['instruction']}
data_df = pd.DataFrame.from_dict(data_dict)
data_df.head()

,explanation_and_answer,answer,input,question
0,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."
1,The statement is well-phrased and easy to foll...,no,### INPUT STORY: My name is Ronald Smith and I...,Is anything in the statement badly phrased or ...
2,The input story does not mention any intention...,yes,### INPUT STORY: My name is Ronald Smith and I...,Does the statement give a different impression...
3,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."
4,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."


In [28]:
data_df["question"][5]

"Is anything in the statement badly phrased or difficult to follow? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement."

In [29]:
### Don't use this anymore
yesnoWhy = " Start by thinking about the question step by step, and include short relevant quotes from the input story and from the statement. Then always end by answering the question concisely with <yes> or <no> being the last word."
yesno_AE = " Start by thinking about the question step by step, and include relevant quotes from the input story and from the statement. Then always end by answering the question concisely with <yes> or <no> being the last word."


diff = " Make sure to think carefully about what is said in the input story VS what is said in the statement."

def modify_question_old(q):
    princ = q.split(" Please answer 'yes' or 'no'.")[0]
    if princ == "Is anything in the statement badly phrased or difficult to follow?":
        princ = "Is anything in the statement badly phrased, too repetative, self-contradictory, or otherwise unreasonable?"
    elif princ == "Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? We don't consider emotional stress or what may happen in the future as suffering.":
        princ = " Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? (Emotional stress should automatically trigger a <yes> here)."
    elif princ == "Does the statement break the rule 'only mention the defendant's age if it is outside of the interval 30-79'?":
        princ = "Does the statement break the rule 'only mention the defendant's age if it is OUTSIDE of the interval 30-79'? Think step by step."

    return princ+yesnoWhy+diff

data_df2 = data_df.copy()
data_df2['question'] = data_df['question'].apply(modify_question_old)


In [30]:
data_df2.head()

,explanation_and_answer,answer,input,question
0,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."
1,The statement is well-phrased and does not con...,no,### INPUT STORY: My name is Ronald Smith and I...,"Is anything in the statement badly phrased, to..."
2,The input story and the statement both acknowl...,yes,### INPUT STORY: My name is Ronald Smith and I...,Does the statement give a different impression...
3,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."
4,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."


In [24]:
json_str = data_df.to_json('data/GPT4_2_half.json', orient = 'records')

##### Sidenote
Cleanup some minor faults; tended to forget a space after end-of-sentence, so instead of generating it all over again, I fixed it w regex:

substituting (\p{L})\.(\p{L}) with $1. $2

#### Only explanations

In [14]:
data_path = "/home/vsb29/rds/hpc-work/project/data/GPT4_2_half.json"
data2 = load_dataset("json", data_files=data_path)
data2['train'][149]['question']

Found cached dataset json (/home/vsb29/.cache/huggingface/datasets/json/default-9a242ef7e101a84e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

"Is anything in the statement badly phrased or difficult to follow? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement."

In [15]:
def modify_questions(q):
    return yesnoCR_turned[yesnoCR_to_int[q.split(" Please answer 'yes' or 'no'")[0]]]

In [21]:
sys_prompt = "You will be presented with a question, and then an input story and a statement. You must analyse the input story and statement in order to properly answer the question."
#yesnoWhy = " Start by thinking about the question step by step, and include short relevant quotes from the input story and from the statement. Then always end by answering the question concisely with <yes> or <no> being the last word."
yesnoWhy = " First, locate quotes from the input story and from the statement that may be relevant to the question. Please think step by step, but do NOT answer the question."

diff = " Make sure to think carefully about what is said in the input story VS what is said in the statement."

#explanations_E = []
f = open("GPT4_explainYN", "a")
for j in range(200,281):
    time.sleep(0.1)
    print(j)
    f.write("\n###"+str(j)+"\n")
    input_story, initial_statement = data2['train'][j]['input'].split("\n### STATEMENT: ")
    question = data2['train'][j]['question'].split(" Please answer 'yes' or 'no'.")[0]
    g4_q = g4_yesno[yesnoCR_to_int[question]] 

    chat_log = [{"role": "system", "content": sys_prompt},
                {"role": "user", "content": "### QUESTION: " + g4_q + yesnoWhy + diff},
                {"role": "user", "content": input_story + "\n### STATEMENT: " + initial_statement + "\n### ANSWER:"}]

    try:
        explanation = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=chat_log,
                temperature=0.5,
                top_p=0.9
            )
    except:
        print("Failed request; we wait.")
        time.sleep(20)
        explanation = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=chat_log,
                temperature=0.5,
                top_p=0.9
            )
    explanations_E.append(explanation['choices'][0]['message']['content'].replace("\n", ""))
    f.write(explanation['choices'][0]['message']['content'].replace("\n", ""))

f.close()

200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
Failed request; we wait.
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


In [22]:
print(len(explanations_E))
explanations_new = explanations_E.copy()

281


In [20]:
# file = open('GPT4_explainYN',mode='r')
# text = file.read()
# explanations_new = text.replace("\n", "").split('###')[1:]

# print(explanations_new[0])
# print(len(explanations_new))

0The input story says, "I am 32 years old and have never done anything like this." and "I got really angry and lost control for a moment." This shows that the speaker acknowledges his actions and feels regret.The statement says, "I am pleading guilty as a mitigating factor in the hope that the court will take this into account when sentencing me." and "I understand the seriousness of my actions and am willing to cooperate with authorities to make things right." This indicates that the speaker is taking responsibility for his actions and is willing to face the consequences.Also, the input story mentions, "I had been drinking with some friends and then this guy said something very rude to my friend," which led to the speaker's actions. The statement also mentions, "On the night in question, I had been drinking with some friends when a guy said something very rude to my friend. In my anger, I lost control and hit him."In both the input story and the statement, the speaker acknowledges tha

In [24]:
full_GPT4 = {'only_explanation': explanations_new, 'explanation_and_answer': data2['train']['explanation_and_answer'], 'answer': data2['train']['answer'], 'input': data2['train']['input'], 'question': data2['train']['question']}
full_GPT4 = pd.DataFrame(full_GPT4)
full_GPT4['question'] = full_GPT4['question'] .apply(modify_questions)

full_GPT4.head(3)

,only_explanation,explanation_and_answer,answer,input,question
0,The input story includes the following relevan...,The statement does not overemphasize or show e...,no,### INPUT STORY: My name is Ronald Smith and I...,"Does the statement overemphasize, or show emot..."
1,The statement seems to be well-phrased and eas...,The statement is well-phrased and easy to foll...,no,### INPUT STORY: My name is Ronald Smith and I...,"Is anything in the statement badly phrased, di..."
2,"The input story says: ""My name is Ronald Smith...",The input story does not mention any intention...,yes,### INPUT STORY: My name is Ronald Smith and I...,Does the statement give a different impression...


In [25]:
full_GPT4['question'][149]

'Is anything in the statement badly phrased, difficult to follow or repetative?'

In [26]:
import re
def remove_starting_numbers(line):
    return re.sub('^\d+', '', line)


In [26]:
#full_GPT4['only_explanation'] = full_GPT4['only_explanation'].apply(remove_starting_numbers)
#full_GPT4.head()

In [27]:
json_str = full_GPT4.to_json('data/GPT4_full3.json', orient = 'records')